<a href="https://colab.research.google.com/github/KaisuH/Emotion-AI/blob/main/Emotion_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotion AI

## Nutshell
As of now this project is under development.

In this project I build a program that detects and localizes cancer from images of human brains, as explained on the course *Modern Artificial Intelligence*, lectured by Dr. Ryan Ahmed, Ph.D. MBA.

The program will train two models which will
1.   classify the images either containing cancer tumor or not
2.   localizes the tumor within hte brain


In [1]:
#HIDE
# @title Loading libraries and setting up drive
%%capture

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Colab Notebooks/brain-tumor-detector/'

# Import the necessary packages
import io
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler, Callback, CSVLogger
from IPython.display import display
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, ZeroPadding2D, BatchNormalization, Activation, Input, Add, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from google.colab.patches import cv2_imshow
import copy
from graphviz import Digraph
from IPython.display import Javascript, HTML, Image, display
import copy
from scipy.ndimage import rotate
import requests

retrain_model = False  # ← Change to True when you want to retrain
rerun_example = False  # ← Change to True when you want to rerun the example
push_git = False # <- Change this to True when you wan to push the changes



In [45]:
#HIDE
push_git = True
# @title GIT commands
# Add changes
if push_git:
  !git add .

In [46]:
#HIDE
%%capture
# Commit changes and remember to change commit message
if push_git:
  !git commit -m "Initial  commit" #Remember to change commit message

In [47]:
!git branch -m main


In [50]:
!git init
!git remote remove origin 2>/dev/null
!git remote add origin https://$token@github.com/KaisuH/brain-tumor-detector.git
!git config user.name  "KaisuH"
!git config user.email "you@example.com"

# 4. Stage & commit all files (including your .gitignore, notebooks, models, etc.)
!git add .
!git commit -m "Initial commit from Colab"

# 5. Rename current branch to main, then push and set upstream
!git branch -M main
!git push -u origin main --force

Reinitialized existing Git repository in /content/drive/MyDrive/Colab Notebooks/brain-tumor-detector/.git/
[main (root-commit) 7f91468] Initial commit from Colab
 2 files changed, 4 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 master.ipynb
fatal: could not read Username for 'https://github.com': No such device or address


In [49]:
!git push -u origin main


error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/KaisuH/brain-tumor-detector.git'


In [44]:
!git push --set-upstream origin main

error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/KaisuH/brain-tumor-detector.git'


In [37]:
!git remote remove origin 2>/dev/null

In [39]:
#HIDE

# Push changes
if push_git:
  with open('/content/drive/MyDrive/tokens/token1.txt', 'r') as f:
      token = f.read().strip()

  !git push --force https://$token@github.com/KaisuH/brain-tumor-detector.git

error: src refspec refs/heads/main does not match any
error: failed to push some refs to 'https://github.com/KaisuH/brain-tumor-detector.git'


In [43]:
!git remote add origin https://$token@github.com/KaisuH/brain-tumor-detector.git main

usage: git remote add [<options>] <name> <url>

    -f, --fetch           fetch the remote branches
    --tags                import all tags and associated objects when fetching
                          or do not fetch any tag at all (--no-tags)
    -t, --track <branch>  branch(es) to track
    -m, --master <branch>
                          master branch
    --mirror[=(push|fetch)]
                          set up remote as a mirror to push to or fetch from



In [6]:
#HIDE
#for removing the last line in gitignore
#!sed -i '$d' .gitignore
#for printing the gitignore
#!cat .gitignore

In [ ]:
make_new_branch = False
if make_new_branch:
  # 1. Create a brand-new “orphan” branch (no history)
  !git checkout --orphan clean-start

  # 2. Stage everything in your current working directory
  !git add -A

  # 3. Commit it as your one “fresh start” commit
  !git commit -m "Fresh start: keep only current work"

  # 4. Force-push this new branch to overwrite remote main
  !git push https://$token@github.com/KaisuH/Emotion-AI.git clean-start:main --force

  # 5. (Optional) Switch back to ‘main’ locally and delete the temp branch
  !git checkout main
  !git branch -D clean-start

In [ ]:
#HIDE
# @title Producing the final file
%%capture
import nbformat

nb_name = "master.ipynb"        # original notebook
out_nb_name = "master_tagged.ipynb"  # new notebook with tags

nb = nbformat.read(nb_name, as_version=4)

for cell in nb.cells:
    if cell.cell_type == 'code':
        # Check if the first line of the cell source is #HIDE
        first_line = cell.source.strip().split('\n',1)[0]
        if first_line.strip().startswith('#HIDE'):
            cell.metadata.setdefault('tags', []).append('hide_input')
nbformat.write(nb, out_nb_name)

In [ ]:
#HIDE
%%capture
#Run this code to generate the tagged file
%%writefile hide_code_config.json
{
  "TagRemovePreprocessor": {
    "enabled": true,
    "remove_input_tags": ["hide_input"]
  },
  "Exporter": {
    "exclude_input_prompt": true,
    "exclude_output_prompt": true
  }
}

In [ ]:
#HIDE
%%capture
# Run this line to produce the output
!jupyter nbconvert --to html --config hide_code_config.json \
    --output "docs/index.html" "master_tagged.ipynb"


### Renaming the document to produce a nicer name in web tab ###
# After nbconvert generates the HTML:
with open("docs/index.html", "r") as f:
    html_content = f.read()

# Find and replace the title tag content
new_title = "Brain tumor detector"  # Your desired title
html_content = html_content.replace("<title>master_tagged</title>", f"<title>{new_title}</title>")

# Insert CSS just before </head> to ensure it loads after default styles
style_snippet = """
<style>
/* Force horizontal scroll on wide code cells */
.jp-RenderedText pre,
.jp-RenderedHTMLCommon pre,
.jp-Cell .jp-Cell-inputWrapper .CodeMirror-lines,
.jp-Cell .jp-Cell-inputWrapper pre {
  white-space: pre !important;
  overflow-x: auto !important;
  max-width: 100% !important;
  display: block !important;
}
</style>
"""

# Insert style snippet at the end of <head>, or if you prefer, after <head>:
html_content = html_content.replace(
    "</head>",
    f"{style_snippet}\n</head>"
)

# Save the modified HTML
with open("docs/index.html", "w") as f:
    f.write(html_content)